In [1]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Using TensorFlow backend.


57344/57026 [==============================] - 0s 4us/step


In [2]:
# 표준화(정규화): 입력 데이터의 특성에 대해 특성의 평균을 빼고 표준편차로 나눈다.
# 상이한 스케일을 가진 값을 신경망에 주입하면 학습이 어려워 진다 - 특성별로 정규화 필요

mean = train_data.mean(axis=0) # 특성의 중앙이 0근처에 맞춰진다
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

# 테스트 데이터를 정규화 할 때 사용한 값이 훈련 데이터에서 계산한 값!

test_data -= mean
test_data /= std

In [5]:
train_data.shape[1], # 13개의 수치 특성

(13,)

In [6]:
# 모델 정의 - 훈련 데이터 적을수록 과대적합 많이 일어남: 더 작은 모델을 사용하는 것이 좋음

from keras import models
from keras import layers

def build_model(): # 동일 모델을 여러 번 사용할 것이기 때문에 함수를 만들어 사용
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape = (train_data.shape[1],)))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

# 스칼라 회귀(하나의 연속적인 값을 예측하는 회귀)를 위한 구성: 마지막 층 하나의 유닛, 활성화 함수 없음
# 마지막 층이 순수한 선형이므로 네트워크가 어떤 범위의 값이라도 예측하도록 자유롭게 학습된다.

In [7]:
# k겹 교차 검증하기 - 데이터 수가 적을 때 데이터를 k개의 분할로 나누고 k개의 모델을 만들어 각각에서 훈련하고 검증해 점수를 낸다.

import numpy as np

k = 4

num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples] # 검증 데이터 준비: k번째 분할
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate( # 훈련 데이터 준비: 다른 분할 전체
    [train_data[:i * num_val_samples],
    train_data[(i+1) * num_val_samples:]],
    axis=0)
    
    partial_train_targets = np.concatenate(
    [train_targets[:i * num_val_samples],
    train_targets[(i+1) * num_val_samples:]],
    axis=0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
             epochs=num_epochs, batch_size=1, verbose=0) # verbose = 0: 훈련과정이 따로 출력되지 않음
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


In [ ]:
# epoch를 500으로 늘리고 각 폴드에서 검증 점수를 로그에 저장하기

num_epochs = 500
all_mae_histories = []

for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples] # 검증 데이터 준비: k번째 분할
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate( # 훈련 데이터 준비: 다른 분할 전체
    [train_data[:i * num_val_samples],
    train_data[(i+1) * num_val_samples:]],
    axis=0)
    
    partial_train_targets = np.concatenate(
    [train_targets[:i * num_val_samples],
    train_targets[(i+1) * num_val_samples:]],
    axis=0)
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0) # verbose = 0: 훈련과정이 따로 출력되지 않음
    
    mae_history = history.history['val_mean_absolute_error'] # val_mean_absolute_error가 history의 key가 아님;;
    all_mae_histories.append(mae_history)

In [ ]:
# k겹 검증 점수 평균을 기록하기

average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

# 검증 점수 그래프

import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history)+1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
# 처음 10개의 데이터 포인트를 제외한 검증 점수 그리기

def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1-factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history)+1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
# 최종 모델 훈련하기

model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)